Предварительно обученное встраивание слов с использованием Glove в модели NLP

GloVe, — это представление слов в виде векторов в непрерывном векторном пространстве, где угол и направление векторов соответствуют семантическим связям между соответствующими словами. Для этого GloVe строит матрицу совместной встречаемости с использованием пар слов, а затем оптимизирует векторы слов, чтобы минимизировать разницу между взаимной информацией по точкам соответствующих слов и скалярным произведением векторов.

Как работает GloVe?

Создание матрицы совместного появления слов является фундаментальным компонентом GloVe. Эта матрица обеспечивает количественную меру семантической близости между словами, фиксируя частоту, с которой они появляются вместе в данном контексте. Затем, минимизируя разницу между скалярным произведением векторов и поточечной взаимной информацией соответствующих слов, GloVe оптимизирует векторы слов. GloVe способен создавать плотные векторные представления, которые фиксируют синтаксические и семантические отношения благодаря своей инновационной методологии.

Создать словарь лексики

Словарь — это набор всех уникальных слов, присутствующих в обучающем наборе данных. Первый набор данных разбивается на слова, затем подсчитывается вся частота каждого слова. Затем слова сортируются в порядке убывания их частот. Слова с высокой частотой помещаются в начало словаря.

Алгоритм встраивания слов

- Предварительная обработка текстовых данных.
- Создал словарь.
- Пройдитесь по файлу перчаток определенного измерения и сравните каждое слово со всеми словами в словаре,
- Если совпадение найдено, скопируйте эквивалентный вектор из перчатки и вставьте в embedding_matrix по соответствующему индексу.

In [ ]:
# code for Glove word embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np
 
x = {'text', 'the', 'leader', 'prime',
     'natural', 'language'}
 
# create the dict.
tokenizer = Tokenizer()
tokenizer.fit_on_texts(x)
 
# number of unique words in dict.
print("Number of unique words in dictionary=", 
      len(tokenizer.word_index))
print("Dictionary is = ", tokenizer.word_index)
 
# download glove and unzip it in Notebook.
#!wget http://nlp.stanford.edu/data/glove.6B.zip
#!unzip glove*.zip
 
# vocab: 'the': 1, mapping of words with
# integers in seq. 1,2,3..
# embedding: 1->dense vector
def embedding_for_vocab(filepath, word_index,
                        embedding_dim):
    vocab_size = len(word_index) + 1
     
    # Adding again 1 because of reserved 0 index
    embedding_matrix_vocab = np.zeros((vocab_size,
                                       embedding_dim))
 
    with open(filepath, encoding="utf8") as f:
        for line in f:
            word, *vector = line.split()
            if word in word_index:
                idx = word_index[word]
                embedding_matrix_vocab[idx] = np.array(
                    vector, dtype=np.float32)[:embedding_dim]
 
    return embedding_matrix_vocab
 
 
# matrix for vocab: word_index
embedding_dim = 50
embedding_matrix_vocab = embedding_for_vocab(
    '../glove.6B.50d.txt', tokenizer.word_index,
  embedding_dim)
 
print("Dense vector for first word is => ",
      embedding_matrix_vocab[1])

Выход:

Количество уникальных слов в словаре = 6 
Словарь = {'leader': 1, 'the': 2, 'prime': 3, 'natural': 4, 'language': 5, 'text': 6} 
Плотный вектор для первого слова => [-0,1567 0,26117 0,78881001 0,65206999 1,20019996 0,35400999 
-0,34298 0,31702 -1,15020001 -0,16099 0,15798 -0,53501999 
-1,34679997 0,51783001 -0,46441001 -0,19846 0,27474999 -0,26154 
  0,25531 0,33388001 -1,04130006 0,52525002 -0,35442999 -0,19137 
-0,08964 -2,33139992 0,12433 -0,94405001 -1,02330005 1,35070002 
  2,55240011 -0,16897 -1,72899997 0,32548001 -0,30914 -0,63056999 
-0,22211 -0,15589 -0,43597999 0,0568 -0,090885 0,75028002 
-1,31529999 -0,75358999 0,82898998 0,051397 -1,48049998 -0,11134 
  0,27090001 -0,48712999]

In [ ]:
import numpy as np
import string
from nltk.corpus import stopwords 
import nltk
nltk.download('stopwords')

# Определение функции Softmax 
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum()

# Создание класса Word2Vec 
class word2vec(object):
    def __init__(self):
        self.N = 10
        self.X_train = []
        self.y_train = []
        self.window_size = 2
        self.alpha = 0.001
        self.words = []
        self.word_index = {}
 
    def initialize(self,V,data):
        self.V = V
        self.W = np.random.uniform(-0.8, 0.8, (self.V, self.N))
        self.W1 = np.random.uniform(-0.8, 0.8, (self.N, self.V))
         
        self.words = data
        for i in range(len(data)):
            self.word_index[data[i]] = i
 
    # Прямое распространение 
    def feed_forward(self,X):
        self.h = np.dot(self.W.T,X).reshape(self.N,1)
        self.u = np.dot(self.W1.T,self.h)
        #print(self.u)
        self.y = softmax(self.u)  
        return self.y

    # Обратное распространение ошибки    
    def backpropagate(self,x,t):
        e = self.y - np.asarray(t).reshape(self.V,1)
        # e.shape is V x 1
        dLdW1 = np.dot(self.h,e.T)
        X = np.array(x).reshape(self.V,1)
        dLdW = np.dot(X, np.dot(self.W1,e).T)
        self.W1 = self.W1 - self.alpha*dLdW1
        self.W = self.W - self.alpha*dLdW

    # Обучение модели  
    def train(self,epochs):
        for x in range(1,epochs):        
            self.loss = 0
            for j in range(len(self.X_train)):
                self.feed_forward(self.X_train[j])
                self.backpropagate(self.X_train[j],self.y_train[j])
                C = 0
                for m in range(self.V):
                    if(self.y_train[j][m]):
                        self.loss += -1*self.u[m][0]
                        C += 1
                self.loss += C*np.log(np.sum(np.exp(self.u)))
            print("epoch ",x, " loss = ",self.loss)
            self.alpha *= 1/( (1+self.alpha*x) )

    # Прогнозирование        
    def predict(self,word,number_of_predictions):
        if word in self.words:
            index = self.word_index[word]
            X = [0 for i in range(self.V)]
            X[index] = 1
            prediction = self.feed_forward(X)
            output = {}
            for i in range(self.V):
                output[prediction[i][0]] = i
             
            top_context_words = []
            for k in sorted(output,reverse=True):
                top_context_words.append(self.words[output[k]])
                if(len(top_context_words)>=number_of_predictions):
                    break
     
            return top_context_words
        else:
            print("Word not found in dictionary")
          

# Предварительная обработка корпуса (очищает и подготавливает текстовые данные, удаляя стоп-слова и знаки препинания, а также преобразуя слова в нижний регистр)
def preprocessing(corpus):
    stop_words = set(stopwords.words('english'))    
    training_data = []
    sentences = corpus.split(".")
    for i in range(len(sentences)):
        sentences[i] = sentences[i].strip()
        sentence = sentences[i].split()
        x = [word.strip(string.punctuation) for word in sentence
                                     if word not in stop_words]
        x = [word.lower() for word in x]
        training_data.append(x)
    return training_data
     
 
def prepare_data_for_training(sentences,w2v):
    data = {}
    for sentence in sentences:
        for word in sentence:
            if word not in data:
                data[word] = 1
            else:
                data[word] += 1
    V = len(data)
    data = sorted(list(data.keys()))
    vocab = {}
    for i in range(len(data)):
        vocab[data[i]] = i
     
    #for i in range(len(words)):
    for sentence in sentences:
        for i in range(len(sentence)):
            center_word = [0 for x in range(V)]
            center_word[vocab[sentence[i]]] = 1
            context = [0 for x in range(V)]
            
            for j in range(i-w2v.window_size,i+w2v.window_size):
                if i!=j and j>=0 and j<len(sentence):
                    context[vocab[sentence[j]]] += 1
                    w2v.X_train.append(center_word)
                    w2v.y_train.append(context)
    w2v.initialize(V,data)
 
    return w2v.X_train,w2v.y_train   

# Запуск обучения и прогнозирования
corpus = ""
corpus += "The earth revolves around the sun. The moon revolves around the earth"
epochs = 1000

training_data = preprocessing(corpus)
w2v = word2vec()

prepare_data_for_training(training_data,w2v)
w2v.train(epochs) 

print(w2v.predict("around",3))    


Определение функции Softmax

Функция softmax используется для преобразования сырых оценок (логитов) в вероятности. Она обычно используется в выходном слое нейронной сети для задач классификации.

Создание класса Word2Vec

Мы определяем word2vecкласс, который будет содержать методы для инициализации весов, выполнения прямого распространения, обратного распространения, обучения и прогнозирования.

Прямое распространение

Метод прямого распространения вычисляет активации скрытого слоя и вероятности выходного слоя с использованием функции softmax.

Обратное распространение ошибки

Метод обратного распространения корректирует веса на основе ошибки между прогнозируемым выходом и фактическими словами контекста. Он вычисляет градиенты и обновляет матрицы весов.

Обучение модели

Метод train итерирует данные обучения для указанного количества эпох. В каждой эпохе он выполняет прямое распространение, обратное распространение и вычисляет потери.

Прогнозирование

Метод прогнозирования берет слово и возвращает верхние контекстные слова на основе обученной модели. Он использует метод прямого распространения для получения вероятностей и сортирует их для поиска наиболее вероятных контекстных слов.

Предварительная обработка корпуса

Функция предварительной обработки очищает и подготавливает текстовые данные, удаляя стоп-слова и знаки препинания, а также преобразуя слова в нижний регистр.

Подготовка данных для обучения

Функция prepare_data_for_training создает обучающие данные путем генерации векторов прямого кодирования для центрального и контекстного слов на основе размера окна.

Запуск обучения и прогнозирования

Наконец, мы запускаем этапы предварительной обработки, обучения и прогнозирования. Мы определяем корпус, проводим его предварительную обработку, готовим данные для обучения, обучаем модель и делаем прогнозы.